In [132]:
## column additions
## elevation change -- ALP - done -----------------------------------
## latitude - longitude distance -- ALP - done ----------------------
## continent - continent code -- ERIC
## country to country code -- SAEID- done----------------------------
## airport to airport code -- ERIC 
## flighttime -- ALP - done -----------------------------------------
## average speed flighttime/distance -- ALP -------------------------
## company name -- ALP - done ---------------------------------------

In [133]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import math

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (11, 7)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna 

# Feel free to add all the libraries you need
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from geopy.distance import geodesic



In [134]:

# Importing airports data and saving it to a csv file
import airportsdata
airports = airportsdata.load('IATA')

# Filling the missing the airport information for SXF with the data of BER
airports['SXF'] = {'icao': 'EDDB',
 'iata': 'SXF',
 'name': 'Berlin Schonefeld Airport',
 'city': 'Berlin',
 'subd': 'Brandenburg',
 'country': 'DE',
 'elevation': 156.0,
 'lat': 52.362167,
 'lon': 13.500667,
 'tz': 'Europe/Berlin',
 'lid': ''}

airports_df = pd.DataFrame(airports).T
airports_df.to_csv('data/airportdata.csv', index=True)

In [135]:

# Import flight data and airport data
df_flightdata = pd.read_csv('data/Train.csv')
df_airportdata = pd.read_csv('data/airportdata.csv',index_col=0)

# # Converting to the right datatype
df_flightdata['STA'] = pd.to_datetime(df_flightdata['STA'], format='%Y-%m-%d %H.%M.%S')
df_flightdata['STD'] = pd.to_datetime(df_flightdata['STD'], format='%Y-%m-%d %H:%M:%S')
df_flightdata['DATOP'] = pd.to_datetime(df_flightdata['DATOP'], format='%Y-%m-%d')


# Extract year, month, and day components
df_flightdata['year'] = df_flightdata['DATOP'].dt.year
df_flightdata['month'] = df_flightdata['DATOP'].dt.month
df_flightdata['day'] = df_flightdata['DATOP'].dt.day


In [136]:
df_flightdata

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,year,month,day
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,ATA,TU 32AIMN,260.0,2016,1,3
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,ATA,TU 31BIMO,20.0,2016,1,13
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,0.0,2016,1,16
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,0.0,2016,1,17
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,ATA,TU 320IMU,22.0,2016,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107828,train_id_107828,2018-07-05,WKL 0000,TUN,TUN,2018-07-05 23:00:00,2018-07-06 02:00:00,SCH,TU 32AIML,0.0,2018,7,5
107829,train_id_107829,2018-01-13,UG 0003,DJE,TUN,2018-01-13 08:00:00,2018-01-13 09:00:00,SCH,UG AT7AT7,0.0,2018,1,13
107830,train_id_107830,2018-11-07,SGT 0000,TUN,TUN,2018-11-07 05:00:00,2018-11-07 12:50:00,SCH,TU 736IOK,0.0,2018,11,7
107831,train_id_107831,2018-01-23,UG 0010,TUN,DJE,2018-01-23 18:00:00,2018-01-23 18:45:00,ATA,TU CR9ISA,0.0,2018,1,23


In [137]:
# Merge df with departure airport data
df = df_flightdata.join(df_airportdata[['iata', 'country', 'elevation', 'lat', 'lon']].add_prefix('dep_'), how='left', on='DEPSTN')
df = df.join(df_airportdata[['iata', 'country', 'elevation', 'lat', 'lon']].add_prefix('arr_'), how='left', on='ARRSTN')
# Merge df with arrival airport data

df

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,...,dep_iata,dep_country,dep_elevation,dep_lat,dep_lon,arr_iata,arr_country,arr_elevation,arr_lat,arr_lon
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,ATA,TU 32AIMN,260.0,...,CMN,MA,656.0,33.367500,-7.58997,TUN,TN,22.0,36.851002,10.22720
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,ATA,TU 31BIMO,20.0,...,MXP,IT,768.0,45.630600,8.72811,TUN,TN,22.0,36.851002,10.22720
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,0.0,...,TUN,TN,22.0,36.851002,10.22720,IST,TR,325.0,41.275333,28.75200
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,0.0,...,DJE,TN,19.0,33.875000,10.77550,NTE,FR,90.0,47.153198,-1.61073
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,ATA,TU 320IMU,22.0,...,TUN,TN,22.0,36.851002,10.22720,ALG,DZ,82.0,36.691002,3.21541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107828,train_id_107828,2018-07-05,WKL 0000,TUN,TUN,2018-07-05 23:00:00,2018-07-06 02:00:00,SCH,TU 32AIML,0.0,...,TUN,TN,22.0,36.851002,10.22720,TUN,TN,22.0,36.851002,10.22720
107829,train_id_107829,2018-01-13,UG 0003,DJE,TUN,2018-01-13 08:00:00,2018-01-13 09:00:00,SCH,UG AT7AT7,0.0,...,DJE,TN,19.0,33.875000,10.77550,TUN,TN,22.0,36.851002,10.22720
107830,train_id_107830,2018-11-07,SGT 0000,TUN,TUN,2018-11-07 05:00:00,2018-11-07 12:50:00,SCH,TU 736IOK,0.0,...,TUN,TN,22.0,36.851002,10.22720,TUN,TN,22.0,36.851002,10.22720
107831,train_id_107831,2018-01-23,UG 0010,TUN,DJE,2018-01-23 18:00:00,2018-01-23 18:45:00,ATA,TU CR9ISA,0.0,...,TUN,TN,22.0,36.851002,10.22720,DJE,TN,19.0,33.875000,10.77550


In [138]:
# Create the seasons column
df.loc[(df['month'] < 3) | (df['month'] == 12), 'season'] = 'winter'
df.loc[(df['month'] >= 3) & (df['month'] < 6), 'season'] = 'spring' 
df.loc[(df['month'] >= 6) & (df['month'] < 9), 'season'] = 'summer' 
df.loc[(df['month'] >= 9) & (df['month'] < 12), 'season'] = 'autumn'

In [139]:
df.isna().sum()
df.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,...,dep_country,dep_elevation,dep_lat,dep_lon,arr_iata,arr_country,arr_elevation,arr_lat,arr_lon,season
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,ATA,TU 32AIMN,260.0,...,MA,656.0,33.367500,-7.58997,TUN,TN,22.0,36.851002,10.22720,winter
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,ATA,TU 31BIMO,20.0,...,IT,768.0,45.630600,8.72811,TUN,TN,22.0,36.851002,10.22720,winter
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,0.0,...,TN,22.0,36.851002,10.22720,IST,TR,325.0,41.275333,28.75200,winter
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,0.0,...,TN,19.0,33.875000,10.77550,NTE,FR,90.0,47.153198,-1.61073,winter
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,ATA,TU 320IMU,22.0,...,TN,22.0,36.851002,10.22720,ALG,DZ,82.0,36.691002,3.21541,winter


In [140]:
# FIX COLUMN NAMES Function - general function build out with time
def fix_column_names(input_dataset):
    infun_data = input_dataset
    # pull the starting columns to show at the end the transformation 
    starting_columns = list(infun_data.columns)
    # carry out the changes; remove spaces and hyphens, lower case everything
    infun_data.columns = infun_data.columns.str.replace(' ','_')
    infun_data.columns = infun_data.columns.str.lower()
    infun_data.columns = infun_data.columns.str.replace('-', '_')
    # pull the ending columns to show at the end the transformation 
    ending_columns = list(infun_data.columns)
    # create a dictionary of the old names vs the new names 
    dict_of_names = {'old column name': starting_columns, 'new column name': ending_columns}
    dataframe_of_column_names = df = pd.DataFrame(dict_of_names) 
    print(dataframe_of_column_names)
    return infun_data

df = fix_column_names(df)

   old column name new column name
0               ID              id
1            DATOP           datop
2            FLTID           fltid
3           DEPSTN          depstn
4           ARRSTN          arrstn
5              STD             std
6              STA             sta
7           STATUS          status
8               AC              ac
9           target          target
10            year            year
11           month           month
12             day             day
13        dep_iata        dep_iata
14     dep_country     dep_country
15   dep_elevation   dep_elevation
16         dep_lat         dep_lat
17         dep_lon         dep_lon
18        arr_iata        arr_iata
19     arr_country     arr_country
20   arr_elevation   arr_elevation
21         arr_lat         arr_lat
22         arr_lon         arr_lon
23          season          season


In [141]:
# you must make the dataset equal to the function
# i.e. df = del_duplicates(df)
def del_duplicates(input_dataset):
    # check for duplicates 
    print(input_dataset.duplicated().value_counts())
    # remove duplicates
    input_dataset = input_dataset.drop_duplicates()
    # reset index inplace
    input_dataset.reset_index(inplace=True, drop=True)
    return input_dataset
    
df = del_duplicates(df)

False    107833
Name: count, dtype: int64


In [142]:
# which are statistical continous values?
# which are statistical quantitative values?


# check the different columns
# is a column a unique identifier? 
# can it be dropped to a unique identifier table, 
# i.e. unique identifier and then name, address, ticket number etc. (info table)
# postcode/zipcode an example of a none unique identifier, things that can't be grouped
df.dtypes

def table_nuniques(input_dataset):
    nunique_dictionary = {'column name': [], 'nuniques': [], 'uniques':[], 'dtype' : []}
    infun_data = input_dataset
    columnlist = list(infun_data.columns)
    for i in columnlist: 
        nunique_dictionary['column name'].append(i)
        nunique_dictionary['nuniques'].append(infun_data[i].nunique())        
        nunique_dictionary['uniques'].append(infun_data[i].unique())
        nunique_dictionary['dtype'].append(infun_data[i].dtype)
    # print the dictionary as a dataframe
    return pd.DataFrame(nunique_dictionary)

table_nuniques(df)

,column name,nuniques,uniques,dtype
0,id,107833,"[train_id_0, train_id_1, train_id_2, train_id_...",object
1,datop,1011,"[2016-01-03 00:00:00, 2016-01-13 00:00:00, 201...",datetime64[ns]
2,fltid,1861,"[TU 0712 , TU 0757 , TU 0214 , TU 0480 , TU 03...",object
3,depstn,132,"[CMN, MXP, TUN, DJE, TLS, IST, ORY, MIR, BRU, ...",object
4,arrstn,128,"[TUN, IST, NTE, ALG, BCN, ORY, FCO, NCE, MRS, ...",object
5,std,81697,"[2016-01-03 10:30:00, 2016-01-13 15:05:00, 201...",datetime64[ns]
6,sta,85136,"[2016-01-03 12:55:00, 2016-01-13 16:55:00, 201...",datetime64[ns]
7,status,5,"[ATA, DEP, RTR, SCH, DEL]",object
8,ac,68,"[TU 32AIMN, TU 31BIMO, TU 736IOK, TU 320IMU, T...",object
9,target,968,"[260.0, 20.0, 0.0, 22.0, 53.0, 10.0, 15.0, 16....",float64


In [143]:
#  no na was found!!
print("Null values")
print(df.isnull().sum())
print("="*50)
print("Zero values")
print((df == 0).sum())

Null values
id               0
datop            0
fltid            0
depstn           0
arrstn           0
std              0
sta              0
status           0
ac               0
target           0
year             0
month            0
day              0
dep_iata         0
dep_country      0
dep_elevation    0
dep_lat          0
dep_lon          0
arr_iata         0
arr_country      0
arr_elevation    0
arr_lat          0
arr_lon          0
season           0
dtype: int64
Zero values
id                   0
datop                0
fltid                0
depstn               0
arrstn               0
std                  0
sta                  0
status               0
ac                   0
target           38168
year                 0
month                0
day                  0
dep_iata             0
dep_country          0
dep_elevation        0
dep_lat              0
dep_lon              0
arr_iata             0
arr_country          0
arr_elevation        0
arr_lat              0
a

In [144]:
# Feature engineering

# Airline names
df['airline_code'] = df['fltid'].str[:2]

# Elevation difference
df['elevation_dif'] = (df['arr_elevation'] - df['dep_elevation'])

# Flight time in minutes
df['flight_time_in_min'] = (df['sta'] - df['std']).dt.total_seconds() / 60

In [145]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,107823,107824,107825,107826,107827,107828,107829,107830,107831,107832
id,train_id_0,train_id_1,train_id_2,train_id_3,train_id_4,train_id_5,train_id_6,train_id_7,train_id_8,train_id_9,...,train_id_107823,train_id_107824,train_id_107825,train_id_107826,train_id_107827,train_id_107828,train_id_107829,train_id_107830,train_id_107831,train_id_107832
datop,2016-01-03 00:00:00,2016-01-13 00:00:00,2016-01-16 00:00:00,2016-01-17 00:00:00,2016-01-17 00:00:00,2016-01-17 00:00:00,2016-01-18 00:00:00,2016-01-18 00:00:00,2016-01-18 00:00:00,2016-01-18 00:00:00,...,2018-04-12 00:00:00,2018-02-09 00:00:00,2018-08-08 00:00:00,2018-10-15 00:00:00,2018-12-19 00:00:00,2018-07-05 00:00:00,2018-01-13 00:00:00,2018-11-07 00:00:00,2018-01-23 00:00:00,2018-11-13 00:00:00
fltid,TU 0712,TU 0757,TU 0214,TU 0480,TU 0338,TU 0283,TU 0514,TU 0716,TU 0752,TU 0996,...,TU 9001,UG 1730,WKL 0000,UG 0011,SGT 0000,WKL 0000,UG 0003,SGT 0000,UG 0010,UG 0002
depstn,CMN,MXP,TUN,DJE,TUN,TLS,TUN,TUN,TUN,TUN,...,MIR,TUN,TUN,DJE,TUN,TUN,DJE,TUN,TUN,TUN
arrstn,TUN,TUN,IST,NTE,ALG,TUN,BCN,ORY,FCO,NCE,...,TUN,NAP,TUN,TUN,TUN,TUN,TUN,TUN,DJE,DJE
std,2016-01-03 10:30:00,2016-01-13 15:05:00,2016-01-16 04:10:00,2016-01-17 14:10:00,2016-01-17 14:30:00,2016-01-17 16:20:00,2016-01-18 07:15:00,2016-01-18 07:35:00,2016-01-18 07:40:00,2016-01-18 07:45:00,...,2018-04-12 17:05:00,2018-02-09 10:00:00,2018-08-08 22:00:00,2018-10-15 19:45:00,2018-12-19 23:45:00,2018-07-05 23:00:00,2018-01-13 08:00:00,2018-11-07 05:00:00,2018-01-23 18:00:00,2018-11-13 06:15:00
sta,2016-01-03 12:55:00,2016-01-13 16:55:00,2016-01-16 06:45:00,2016-01-17 17:00:00,2016-01-17 15:50:00,2016-01-17 18:15:00,2016-01-18 09:00:00,2016-01-18 09:55:00,2016-01-18 09:00:00,2016-01-18 09:15:00,...,2018-04-12 17:40:00,2018-02-09 11:15:00,2018-08-09 01:00:00,2018-10-15 20:45:00,2018-12-20 01:45:00,2018-07-06 02:00:00,2018-01-13 09:00:00,2018-11-07 12:50:00,2018-01-23 18:45:00,2018-11-13 07:05:00
status,ATA,ATA,ATA,ATA,ATA,ATA,ATA,ATA,ATA,ATA,...,ATA,SCH,SCH,SCH,SCH,SCH,SCH,SCH,ATA,SCH
ac,TU 32AIMN,TU 31BIMO,TU 32AIMN,TU 736IOK,TU 320IMU,TU 736IOP,TU 32AIMH,TU 32AIMI,TU 32AIMC,TU 31AIMK,...,TU 736IOL,TU CR9ISA,TU 320IMV,UG AT7LBE,TU 736IOP,TU 32AIML,UG AT7AT7,TU 736IOK,TU CR9ISA,TU CR9ISA
target,260.0,20.0,0.0,0.0,22.0,53.0,10.0,15.0,16.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
# Distance between departure and arrival airports using the library geodesic
def calculate_distance(row):
    dep_coords = (row['dep_lat'], row['dep_lon'])
    arr_coords = (row['arr_lat'], row['arr_lon'])
    distance = geodesic(dep_coords, arr_coords).kilometers
    return int(round(distance,0))

df['flight_distance_in_km'] = df.apply(calculate_distance, axis=1)

In [147]:
df['average_flight_speed_km_h'] = (df['flight_distance_in_km']*60/df['flight_time_in_min'])
df['average_flight_speed_km_h'] = df['average_flight_speed_km_h'].round().astype(int)


In [148]:
df['international_flight'] = np.where(df['arr_country'] != df['dep_country'], 1, 0)

In [149]:
df['international_flight'].value_counts()

international_flight
1    88932
0    18901
Name: count, dtype: int64

In [150]:
df.head().T

,0,1,2,3,4
id,train_id_0,train_id_1,train_id_2,train_id_3,train_id_4
datop,2016-01-03 00:00:00,2016-01-13 00:00:00,2016-01-16 00:00:00,2016-01-17 00:00:00,2016-01-17 00:00:00
fltid,TU 0712,TU 0757,TU 0214,TU 0480,TU 0338
depstn,CMN,MXP,TUN,DJE,TUN
arrstn,TUN,TUN,IST,NTE,ALG
std,2016-01-03 10:30:00,2016-01-13 15:05:00,2016-01-16 04:10:00,2016-01-17 14:10:00,2016-01-17 14:30:00
sta,2016-01-03 12:55:00,2016-01-13 16:55:00,2016-01-16 06:45:00,2016-01-17 17:00:00,2016-01-17 15:50:00
status,ATA,ATA,ATA,ATA,ATA
ac,TU 32AIMN,TU 31BIMO,TU 32AIMN,TU 736IOK,TU 320IMU
target,260.0,20.0,0.0,0.0,22.0


In [151]:
df

,id,datop,fltid,depstn,arrstn,std,sta,status,ac,target,...,arr_elevation,arr_lat,arr_lon,season,airline_code,elevation_dif,flight_time_in_min,flight_distance_in_km,average_flight_speed_km_h,international_flight
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12:55:00,ATA,TU 32AIMN,260.0,...,22.0,36.851002,10.22720,winter,TU,-634.0,145.0,1667,690,1
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16:55:00,ATA,TU 31BIMO,20.0,...,22.0,36.851002,10.22720,winter,TU,-746.0,110.0,983,536,1
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,0.0,...,325.0,41.275333,28.75200,winter,TU,303.0,155.0,1673,648,1
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,0.0,...,90.0,47.153198,-1.61073,winter,TU,71.0,170.0,1805,637,1
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15:50:00,ATA,TU 320IMU,22.0,...,82.0,36.691002,3.21541,winter,TU,60.0,80.0,626,470,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107828,train_id_107828,2018-07-05,WKL 0000,TUN,TUN,2018-07-05 23:00:00,2018-07-06 02:00:00,SCH,TU 32AIML,0.0,...,22.0,36.851002,10.22720,summer,WK,0.0,180.0,0,0,0
107829,train_id_107829,2018-01-13,UG 0003,DJE,TUN,2018-01-13 08:00:00,2018-01-13 09:00:00,SCH,UG AT7AT7,0.0,...,22.0,36.851002,10.22720,winter,UG,3.0,60.0,334,334,0
107830,train_id_107830,2018-11-07,SGT 0000,TUN,TUN,2018-11-07 05:00:00,2018-11-07 12:50:00,SCH,TU 736IOK,0.0,...,22.0,36.851002,10.22720,autumn,SG,0.0,470.0,0,0,0
107831,train_id_107831,2018-01-23,UG 0010,TUN,DJE,2018-01-23 18:00:00,2018-01-23 18:45:00,ATA,TU CR9ISA,0.0,...,19.0,33.875000,10.77550,winter,UG,-3.0,45.0,334,445,0


In [152]:
# Convert timestamps to numeric features
df['std_time'] = df['std'].dt.time
df['sta_time'] = df['sta'].dt.time

In [153]:
df['std_time'] = df['std_time'].astype(str).str.replace(':', '').astype(int)
df['sta_time'] = df['std_time'].astype(str).str.replace(':', '').astype(int)
df['datop'] = df['datop'].astype(str).str.replace('-', '').astype(int)

In [154]:
table_nuniques(df)

,column name,nuniques,uniques,dtype
0,id,107833,"[train_id_0, train_id_1, train_id_2, train_id_...",object
1,datop,1011,"[20160103, 20160113, 20160116, 20160117, 20160...",int32
2,fltid,1861,"[TU 0712 , TU 0757 , TU 0214 , TU 0480 , TU 03...",object
3,depstn,132,"[CMN, MXP, TUN, DJE, TLS, IST, ORY, MIR, BRU, ...",object
4,arrstn,128,"[TUN, IST, NTE, ALG, BCN, ORY, FCO, NCE, MRS, ...",object
5,std,81697,"[2016-01-03 10:30:00, 2016-01-13 15:05:00, 201...",datetime64[ns]
6,sta,85136,"[2016-01-03 12:55:00, 2016-01-13 16:55:00, 201...",datetime64[ns]
7,status,5,"[ATA, DEP, RTR, SCH, DEL]",object
8,ac,68,"[TU 32AIMN, TU 31BIMO, TU 736IOK, TU 320IMU, T...",object
9,target,968,"[260.0, 20.0, 0.0, 22.0, 53.0, 10.0, 15.0, 16....",float64


In [155]:
eda_data = df.drop(['id', 'std', 'sta'], axis=1)

In [156]:
eda_data

,datop,fltid,depstn,arrstn,status,ac,target,year,month,day,...,arr_lon,season,airline_code,elevation_dif,flight_time_in_min,flight_distance_in_km,average_flight_speed_km_h,international_flight,std_time,sta_time
0,20160103,TU 0712,CMN,TUN,ATA,TU 32AIMN,260.0,2016,1,3,...,10.22720,winter,TU,-634.0,145.0,1667,690,1,103000,103000
1,20160113,TU 0757,MXP,TUN,ATA,TU 31BIMO,20.0,2016,1,13,...,10.22720,winter,TU,-746.0,110.0,983,536,1,150500,150500
2,20160116,TU 0214,TUN,IST,ATA,TU 32AIMN,0.0,2016,1,16,...,28.75200,winter,TU,303.0,155.0,1673,648,1,41000,41000
3,20160117,TU 0480,DJE,NTE,ATA,TU 736IOK,0.0,2016,1,17,...,-1.61073,winter,TU,71.0,170.0,1805,637,1,141000,141000
4,20160117,TU 0338,TUN,ALG,ATA,TU 320IMU,22.0,2016,1,17,...,3.21541,winter,TU,60.0,80.0,626,470,1,143000,143000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107828,20180705,WKL 0000,TUN,TUN,SCH,TU 32AIML,0.0,2018,7,5,...,10.22720,summer,WK,0.0,180.0,0,0,0,230000,230000
107829,20180113,UG 0003,DJE,TUN,SCH,UG AT7AT7,0.0,2018,1,13,...,10.22720,winter,UG,3.0,60.0,334,334,0,80000,80000
107830,20181107,SGT 0000,TUN,TUN,SCH,TU 736IOK,0.0,2018,11,7,...,10.22720,autumn,SG,0.0,470.0,0,0,0,50000,50000
107831,20180123,UG 0010,TUN,DJE,ATA,TU CR9ISA,0.0,2018,1,23,...,10.77550,winter,UG,-3.0,45.0,334,445,0,180000,180000


In [157]:

eda_data.to_csv('data/eda_data.csv', index=True)